In [ ]:
import os
from datetime import datetime

import pandas as pd

from eo_flood_ops.model_utils import GroundTruthMeasurement
from eo_flood_ops.thresholding_model import ThresholdingModel 
from eo_flood_ops.general_utils import tif_to_clipped_masked_array, find_closest_valid, generate_wet_dry_timeseries_ds

class MyGroundTruthMeasurement(GroundTruthMeasurement):
  pass

In [2]:
# Directories and filenames
images_dir = R"p:\11211461-010--eo-flood-ops\HoiAn\classified_events"
aoi_dir = R"p:\11211461-010--eo-flood-ops\HoiAn\AOI"
waterlevels_fn = R"p:\11211461-010--eo-flood-ops\HoiAn\water_levels\Water_Level.csv"
dem_fn = R"p:\11211461-010--eo-flood-ops\HoiAn\DEMs\hoi_an_FABDEM.tif"

# Load water level data
df_water = pd.read_csv(waterlevels_fn, skiprows=[1])
df_water.rename(columns={df_water.columns[0]: "datetime"}, inplace=True)
df_water["datetime"] = pd.to_datetime(df_water["datetime"])

prepare datasets for training

In [3]:
results_per_aoi = {}

# Loop over AOI files
for aoi_file in os.listdir(aoi_dir):
    if not aoi_file.endswith('.geojson'):
        continue
    
    aoi_name = aoi_file.split('_')[1].split('.')[0]

    print(f"Processing AOI: {aoi_name}")
    print("*" * 45)
    # Store image-water level tuples
    image_water_levels = []
    
    # Loop over images
    for image_name in os.listdir(images_dir):
        if not image_name.endswith(".tif"):
            continue
        

        datetime_str = image_name.split("_")[1].replace(".tif", "") 
        dt = datetime.strptime(datetime_str, "%Y%m%dT%H%M%S")
        dt = pd.Timestamp(dt)
        dt_gmt7 = dt + pd.Timedelta(hours=7)  # Convert to GMT+7

        closest_time, value = find_closest_valid(df_water, dt_gmt7, aoi_name)

        if closest_time is not None:
            print(f"Image: {image_name}")
            print(f"Image timestamp (GMT+7): {dt_gmt7}")
            print("Closest valid timestamp:", closest_time)
            print(f"{aoi_name} value:", value)
            print("=" * 45)
            image_water_levels.append((image_name, value))
        else:
            print(f"No valid {aoi_name} values found in dataset for image {image_name}.")

    if not image_water_levels:
        print(f"No images found for AOI: {aoi_name}")
        continue

    # print some spacing to improve readability
    print("\n" * 1)

    # Sort by water level ascending
    image_water_levels.sort(key=lambda x: x[1])
    
    # Store results
    results_per_aoi[aoi_name] = image_water_levels

Processing AOI: ID6
*********************************************
Image: hoian_20221015T105600.tif
Image timestamp (GMT+7): 2022-10-15 17:56:00
Closest valid timestamp: 2022-10-15 18:00:00
ID6 value: 3.25
Image: hoian_20221202T105600.tif
Image timestamp (GMT+7): 2022-12-02 17:56:00
Closest valid timestamp: 2022-12-02 18:00:00
ID6 value: 1.29
Image: hoian_20221206T223600.tif
Image timestamp (GMT+7): 2022-12-07 05:36:00
Closest valid timestamp: 2022-12-07 05:40:00
ID6 value: 1.31
Image: hoian_20231115T105600.tif
Image timestamp (GMT+7): 2023-11-15 17:56:00
Closest valid timestamp: 2023-11-15 18:00:00
ID6 value: 2.06




In [4]:
# Prepare GROUND TRUTH for each AOI

# Dictionary to store GROUND_TRUTH per AOI
GROUND_TRUTH_per_aoi = {}

for aoi_file in os.listdir(aoi_dir):
    if not aoi_file.endswith('.geojson'):
        continue
    
    aoi_name = aoi_file.split('_')[1].split('.')[0]
    if aoi_name == "Modrica":  # skip for now
        continue
    
    aoi_fn = os.path.join(aoi_dir, aoi_file)
    
    # Make sure we have results for this AOI
    if aoi_name not in results_per_aoi:
        print(f"No images found for AOI: {aoi_name}")
        continue
    
    GROUND_TRUTH = []
    
    for image_name, water_level in results_per_aoi[aoi_name]:
        image_path = os.path.join(images_dir, image_name)
        
        masked_array, transforms, crs = tif_to_clipped_masked_array(
            image_path,
            aoi_fn,
        )
        
        GROUND_TRUTH.append(MyGroundTruthMeasurement(
            ground_truth=masked_array,
            gauge_measurement=water_level
        ))
    
    # Store per AOI
    GROUND_TRUTH_per_aoi[aoi_name] = GROUND_TRUTH

In [5]:
# MIN_RATIOS = [0.1, 0.3, 0.5, 1, 2, 5, 10, 15, 20]
MIN_RATIOS = [0.5, 1]

trained_models_tm= {}

for aoi_name, GROUND_TRUTH in GROUND_TRUTH_per_aoi.items():
    print("\n" + "="*60)
    print(f"Starting training for AOI: {aoi_name}")
    print(f"Number of ground truth images: {len(GROUND_TRUTH)}")
    print(f"Using minimum ratios: {MIN_RATIOS}")
    
    tm = ThresholdingModel()
    
    tm.train(
        minumum_ratios=MIN_RATIOS,
        ground_truth=GROUND_TRUTH
    )
    
    trained_models_tm[aoi_name] = tm
    print(f"Finished training for AOI: {aoi_name}")
    print("="*60)


Starting training for AOI: ID6
Number of ground truth images: 4
Using minimum ratios: [0.5, 1]
For min_ratio=0.5 we get f1=0.9615551717575752
For min_ratio=1 we get f1=0.9651129127404915
chosen min_ratio 1
Finished training for AOI: ID6


In [10]:
# example use of the function to generate a time series dataset
gauge_levels = [1, 6, 7.5, 3.2, 10]
timestamps = pd.date_range("2020-01-01", periods=len(gauge_levels), freq="7D")

ds = generate_wet_dry_timeseries_ds(gauge_levels, timestamps, tm, transforms, crs)
ds


<xarray.Dataset> Size: 31MB
Dimensions:      (lat: 1163, lon: 1317, time: 5)
Coordinates:
  * lat          (lat) float64 9kB 15.77 15.77 15.77 15.77 ... 16.08 16.08 16.08
  * lon          (lon) float64 11kB 108.0 108.0 108.0 ... 108.4 108.4 108.4
  * time         (time) datetime64[ns] 40B 2020-01-01 2020-01-08 ... 2020-01-29
    water_level  (time) float64 40B 1.0 6.0 7.5 3.2 10.0
Data variables:
    wet_dry      (time, lat, lon) float32 31MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    spatial_ref  int64 8B 0

In [11]:
ds

<xarray.Dataset> Size: 31MB
Dimensions:      (lat: 1163, lon: 1317, time: 5)
Coordinates:
  * lat          (lat) float64 9kB 15.77 15.77 15.77 15.77 ... 16.08 16.08 16.08
  * lon          (lon) float64 11kB 108.0 108.0 108.0 ... 108.4 108.4 108.4
  * time         (time) datetime64[ns] 40B 2020-01-01 2020-01-08 ... 2020-01-29
    water_level  (time) float64 40B 1.0 6.0 7.5 3.2 10.0
Data variables:
    wet_dry      (time, lat, lon) float32 31MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    spatial_ref  int64 8B 0